<a href="https://colab.research.google.com/github/ashishsarkarmlops/Text-generation-LSTM/blob/main/HACKANONS_COLAB_25GB_RAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [1]:
import numpy as np 
import pandas as pd 
%tensorflow_version 2.x
import tensorflow 
import requests
import string

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [2]:
response=requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [3]:
corpus=response.text

In [4]:
data=corpus.split('\n')

In [5]:
data=data[253:]

In [6]:
len(data)

124204

In [7]:
data=" ".join(data)

In [8]:
data=data.lower()

Clean the data

In [9]:

#cleaning functions 
def clean_text(doc):
  tokens=doc.split()
  table=str.maketrans('','', string.punctuation)
  #taking not punctuations
  tokens = [w.translate(table) for w in tokens]
  #taking alphabets only
  tokens=[word for word in tokens if word.isalpha()]
  return tokens
     

In [10]:
tokens=clean_text(data)
#print(tokens[:50])

In [11]:
#overall words
len(tokens)
     

898199

In [12]:
#unique words 
len(set(tokens))
     

27956

The method we will use is well take 50 words to predict the 51st word

In [14]:
#we will use 50 words to predict the 51 word +1 will be the target variable 
length=50+1

In [15]:
lines=[]
for i in range (length,len(tokens)):
  seq=tokens[i-length:i]
  line=' '.join(seq)
  lines.append(line)
  if i> 200000:
   break
print(len(lines))

199951


In [16]:
#lines[0]

In [17]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [18]:
#tokenizer sentences 
tokenizer=Tokenizer()

In [19]:
tokenizer.fit_on_texts(lines)
sequences=tokenizer.texts_to_sequences(lines)

In [20]:
sequences=np.array(sequences)

In [21]:
X=sequences[:,:-1]
y=sequences[:,-1]

In [22]:
vocab_size=len(tokenizer.word_index)+1 # this gives the index for each word   =>tokenizer.word_index

In [23]:
y=to_categorical(y,num_classes=vocab_size)# converting to one hot encoded matrix 

In [24]:
seq_length=X.shape[1]

In [25]:
#Building LSTM Model
model=Sequential()

In [26]:
model.add(Embedding(vocab_size,50,input_length=seq_length))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100,activation='relu'))
model.add(Dense(vocab_size,activation='softmax'))

In [27]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            650450    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 13009)             1313909   
                                                                 
Total params: 2,115,259
Trainable params: 2,115,259
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics='accuracy')

In [ ]:
model.fit(X,y,epochs=100,batch_size=256)